# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
# Setup of the workspace
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException
# load Dataset
from train import read_data
# AzureML run
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
# Deploy the model
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, LocalWebservice
# score the model
import json, requests

## Dataset

### Overview
The dataset I'm using for this project is the Heart Failure Prediction Dataset from kaggle.

fedesoriano. (September 2021). Heart Failure Prediction Dataset. Retrieved [2021-10-18] from https://www.kaggle.com/fedesoriano/heart-failure-prediction.

The task with this dataset is a classification task to predict whether a person will develop a heart disease with a set of 11 diagnostic features. 
This dataset is a combination of five independent heart disease datasets containing 918 observations of patients. The target column "Heart Disease" is nearly balanced in this dataset with 510 patients with and 408 patients without cardiovascular diseases.

The features in this dataset are:

General information
- Age of the patient in years; with the youngest patient being 28 and the oldest 77 years old
- Sex of the patient; the majority of the patients (79%) being male

blood tests
- Cholesterol in serum $[$mm/dl$]$; an indicator for ateriosclerosis
- fasting blood sugar level; a boolean value if the blood sugar is elevated (>120mg/dl) or not, which is an indicator for diabetes

medical history
- type of chest pain the patient is experiencing in four categories (TA: typical angina, ATA: atypical angina, NAP: non-anginal pain, ASY: asymptomatic)

ECG and cardiac stress testing
- resting blood pressure $[$mm Hg$]$
- resting ECG results in three categories (Normal, ST: ST-T wave abnormality, LVH : left ventricular hypertrophy)
- oldpeak: the depression between S and T peak
- maximum heartrate under cardiac stress
- exercise-induced angina
- ST-slope of te peak exercise in three categories (Up, flat and down)


### Setup workspace and experiment

Use Workspace.from_config() to get the workspace configuration in the VM.
Set up an experiment with the name "heart-failure-experiment".

In [2]:
ws = Workspace.from_config()

# print some information about the workspace
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# choose a name for experiment
experiment_name = 'heart-failure-experiment'

experiment=Experiment(ws, experiment_name)

Workspace name: quick-starts-ws-161946
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-161946


### Create a cluster

In [3]:
cluster_name = "expcluster"

# Use existing cluster, if it exists
try:
    compute_target = ComputeTarget(workspace=ws, name = cluster_name)
    print('Found existing cluster, use it!')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',
                                                          max_nodes=6, min_nodes=1)
    compute_target = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)
compute_target.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Load Dataset

Since I need the dataset for the AutoML run as well as for the HyperDriveRun I configured the upload of local data and registering of the dataset in the function `read_data()` from the train.py script. This function also calls the `prepare_data()` function in the script, which preprocesses and cleans the dataset

In [7]:
dataset=read_data()

Uploading an estimated of 3 files
Uploading ./data/.amlignore
Uploaded ./data/.amlignore, 1 files out of an estimated total of 3
Uploading ./data/.amlignore.amltmp
Uploaded ./data/.amlignore.amltmp, 2 files out of an estimated total of 3
Uploading ./data/heart.csv
Uploaded ./data/heart.csv, 3 files out of an estimated total of 3
Uploaded 3 files
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/6a7dea94-c39b-4548-88f8-ca456ae79c60/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [8]:
# check whether the dataset is loaded correctly
df = dataset.to_pandas_dataframe()
df.describe()

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,0.789760,132.396514,198.799564,0.233115,136.809368,0.404139,0.887364,0.361656,0.553377,0.540305,0.188453,0.221133,0.050109,0.204793,0.601307,0.193900
std,9.432617,0.407701,18.514154,109.384145,0.423046,25.460334,0.490992,1.066570,0.607056,0.497414,0.498645,0.391287,0.415236,0.218289,0.403770,0.489896,0.395567
min,28.000000,0.000000,0.000000,0.000000,0.000000,60.000000,0.000000,-2.600000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.000000,1.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,54.000000,1.000000,130.000000,223.000000,0.000000,138.000000,0.000000,0.600000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,60.000000,1.000000,140.000000,267.000000,0.000000,156.000000,1.000000,1.500000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,77.000000,1.000000,200.000000,603.000000,1.000000,202.000000,1.000000,6.200000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## AutoML Configuration

For the automl_settings I used a timeout of 20minutes, to ensure completion of the experiment before the VM times out.
The `maximum concurrent_iterations` are 5, meaning that five iterations can run in parallel. This is less than the maximum number of nodes of my compute_cluster. The primary metric to evaluate the best model is `accuracy`. This is the most straightforwar metric to judge a classification task. Since the target values are quite balanced in the dataset, I dont expect much problems with the precision of the resulting model. Instead of splitting the dataset into training and test data, I use a 5-fold crossvalidation. The early stooping logic is enabled, which stops an iteration if there is no improvement in the primary metric after 31 iterations.

I configured the AutoMl run, that it runs on the above specified compute cluster ("compute_target").
The AutoML experiment will run a classification task on the heart-failure-dataset with the target column "HeartDisease".
I turned the featurization off, since the data is already preprocessed. I enabled the export to onnx format with `enable_onnx_compatible_models`.

In [9]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric": 'accuracy',
    "n_cross_validations": 5,
    "enable_early_stopping": True,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task="classification",
                             training_data=dataset,
                             label_column_name="HeartDisease",
                             path='./automl_run',
                             featurization = "off",
                             enable_onnx_compatible_models=True,
                             **automl_settings
                            )

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-experiment,AutoML_c21c4484-db3d-4079-a41a-e6c8cb339fb5,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

In [11]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

##### OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

In the AutoML run different types of models were used: `LogisticRegression`, nearest-neighbours-type models (`KNN`, `SVM`), and tree-based learning algorithms (`XGBoostClassifiers`, `LightGBM`, `ExtremeRandomTrees`, `RandomForest`).
The tree-based algorithms seem to perform better than the nearest-neighbour-types. This might be due to the fact, that most of the features in my training data is binary and only five features are numerical.

AutoML chose the `manhatten metric` as distance metric for the `KNN`and `SVM` models. This metric works good on standard normal or standard uniform distributions of numeric features. But in this dataset, the distribution of the features is more like a Bernoulli distribution, which is not optimal for this metric and results in the lower accuracy scores of the models.

However binary features make a distinction between leaves easy for the algorithm. And with a balanced dataset, the resulting tree models won't suffer much bias.

In [15]:
remote_run.summary()

[['Failed', 3, nan],
 ['LightGBM', 5, 0.8649204086481349],
 ['XGBoostClassifier', 16, 0.8725706818721786],
 ['KNN', 6, 0.8562128771679735],
 ['ExtremeRandomTrees', 1, 0.8463827512473271],
 ['LogisticRegression', 5, 0.8572998336897125],
 ['SVM', 1, 0.8366060822047994],
 ['RandomForest', 5, 0.8747148966500358],
 ['VotingEnsemble', 1, 0.8878118317890236]]

## Best Model

The best model of the AutoML run is a VotingEnsemble containing mainly XGBoostClassifier models, LightGBM, RandomForest and KNN models.
It has an accuracy of `0.883` and a precision of `0.889`. The model reliability is ~15% False-positive and ~8% False-Negative predictions (according to the confusion matrix).

In [16]:
best_automl_run = remote_run.get_best_child()
print("best run details: ", best_automl_run.get_details())
print("best run metrics: ", best_automl_run.get_metrics())

best run details:  {'runId': 'AutoML_c21c4484-db3d-4079-a41a-e6c8cb339fb5_40', 'target': 'expcluster', 'status': 'Completed', 'startTimeUtc': '2021-10-25T11:29:23.301665Z', 'endTimeUtc': '2021-10-25T11:30:09.299041Z', 'services': {}, 'warnings': [{'message': 'This run is using a new job runtime with improved performance and error reporting. The logs from your script are in user_logs/std_log.txt. Please let us know if you run into any issues, and if you would like to opt-out, please add the environment variable AZUREML_COMPUTE_USE_COMMON_RUNTIME to the environment variables section of the job and set its value to the string "false"'}], 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\

In [17]:
print(best_automl_run.get_file_names())

['accuracy_table', 'automl_driver.py', 'confusion_matrix', 'explanation/90e384c7/classes.interpret.json', 'explanation/90e384c7/eval_data_viz.interpret.json', 'explanation/90e384c7/expected_values.interpret.json', 'explanation/90e384c7/features.interpret.json', 'explanation/90e384c7/global_names/0.interpret.json', 'explanation/90e384c7/global_rank/0.interpret.json', 'explanation/90e384c7/global_values/0.interpret.json', 'explanation/90e384c7/local_importance_values.interpret.json', 'explanation/90e384c7/per_class_names/0.interpret.json', 'explanation/90e384c7/per_class_rank/0.interpret.json', 'explanation/90e384c7/per_class_values/0.interpret.json', 'explanation/90e384c7/rich_metadata.interpret.json', 'explanation/90e384c7/true_ys_viz.interpret.json', 'explanation/90e384c7/visualization_dict.interpret.json', 'explanation/90e384c7/ys_pred_proba_viz.interpret.json', 'explanation/90e384c7/ys_pred_viz.interpret.json', 'explanation/eba0c904/classes.interpret.json', 'explanation/eba0c904/eva

In [18]:
#TODO: Save the best model in pickel and onnx format
for f in best_automl_run.get_file_names():
    if f.startswith('outputs/model.onnx'):
        output_file_path = os.path.join('./automl_model', 'automl_model.onnx')
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_automl_run.download_file(name=f, output_file_path=output_file_path)
    elif f.startswith('outputs/model.pkl')
        output_file_path = os.path.join('./automl_model', 'automl_model.pkl')
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_automl_run.download_file(name=f, output_file_path=output_file_path)

## Model Deployment
### Register Model

In [19]:
model_name = best_automl_run.properties['model_name']
description = "AutoML heart-failure classification model"

model = remote_run.register_model(model_name = model_name,
                                  description = description)

In [20]:
# Registering the ONNX Model
model_onnx = Model.register(workspace=ws,
    model_name='automl_onnx_model',
    model_path='./automl_model/automl_model.onnx',
    model_framework=Model.Framework.ONNX,
    model_framework_version='1.3',
    description=description)

Registering model AutoMLc21c4484d40


In [21]:
# get environment yml and scoring scripts for the pkl model
for f in best_automl_run.get_file_names():
    if f.__contains__("scoring"):
        output_file_path = os.path.join('./automl_model', os.path.basename(f))
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_automl_run.download_file(name=f, output_file_path=output_file_path)
    if f.__contains__("conda_env"):
        output_file_path = os.path.join('./automl_model', os.path.basename(f))
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_automl_run.download_file(name=f, output_file_path=output_file_path)


To deploy a model an inference_config is needed.
First I need to define the environment for the inference cluster. In the AutoML run a `yml` file was automatically created, which I downloaded. Second an entry script for the model is also created in the AutoML run.

In [23]:
service_name = 'heart-failure-automl-service'

env = Environment.from_conda_specification(name="automl_env", file_path='./automl_model/conda_env_v_1_0_0.yml')
inference_config = InferenceConfig(entry_script='automl_model/scoring_file_v_1_0_0.py',
                                   environment=env)

### Local deployment
First deploy the model locally to test, if everything works

In [ ]:
# deploy to local for debugging
deployment_config = LocalWebservice.deploy_configuration(port=6789)
test_service = Model.deploy(
    ws,
    name='test-service',
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config,
    overwrite=True
)
test_service.wait_for_deployment(show_output=True)

In [ ]:
# get some testdata to send a request
data = df.head(1).drop("HeartDisease", axis=1).to_json(orient='records')
body = {"data": data,
       "method": "predict"}
print(body)

In [61]:
# test against local deployment
uri = test_service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
response = requests.post(uri, data=json.dumps(body), headers=headers)
print(response.json())

{"result": [0]}


In [ ]:
# local deployment is working, it can be deleted
test_service.delete()

### Deployment ot Webservice
After testing the local deployment, I deploy the model to an Azure Container Instance with 1 CPU core and 1GB Memory. I enabled authentification and app_insights for the web service.

In [28]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,
                                                enable_app_insights=True,
                                                auth_enabled=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-21 15:41:40+00:00 Creating Container Registry if not exists..
2021-10-21 15:51:41+00:00 Registering the environment.
2021-10-21 15:51:43+00:00 Building image..
2021-10-21 16:03:29+00:00 Generating deployment configuration..
2021-10-21 16:03:30+00:00 Submitting deployment to compute.
2021-10-21 16:03:34+00:00 Checking the status of deployment heart-failure-automl-service..
2021-10-21 16:08:44+00:00 Checking the status of inference endpoint heart-failure-automl-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


The url for the swagger documentation of the REST Endpoint of this model can be found using the method `swagger_uri` of the Webservice object.
To consume the model, I need the scoring uri and (since it is an ACI) a key to authentificate my request. I get those using the `scoring_uri`and `get_keys()` methods of the Webservice object.

In [62]:
# send request to deployed web service
uri = service.scoring_uri
print(uri)
print(service.swagger_uri)
key, _ = service.get_keys()

http://05985b95-0713-4bf4-a765-dbe4ba7f04ee.southcentralus.azurecontainer.io/score
http://05985b95-0713-4bf4-a765-dbe4ba7f04ee.southcentralus.azurecontainer.io/swagger.json


In [63]:
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"
response = requests.post(uri, data=json.dumps(body), headers=headers)
print(response.json())

{"result": [0]}


### Service Logs

In [30]:
print(service.get_logs())

2021-10-21T16:08:36,868490900+00:00 - iot-server/run 
2021-10-21T16:08:36,898753100+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-10-21T16:08:36,905560200+00:00 - rsyslog/run 
2021-10-21T16:08:36,943083600+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_6e432c07894b32e6709e117e3d3d4688/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-10-21T16:08:37,519733000+00:00 - iot-server/finish 1 0
2021-10-21T16:08:37,526618400+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (70)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 94
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-10-21 16:08:40,074 | root | INFO | Starting up app insights client
logging socket was 

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
